In [5]:
import os, pandas as pd
import numpy as np
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [6]:
parent_directory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
main_directory = os.path.dirname(parent_directory)
phase_1 =  pd.read_csv(main_directory + '/data/phase_1_data_with_survey.csv')
phase_1.head()

,ID,From,Date,Hour,Participation_Phase,Demand_kWh,Temperature,home_size,electric_car,no_of_people,electrically_heated
0,Exp_62,2020-02-01 00:00:00+00:00,2020-02-01,1,Phase_1,1.925,7.6,120-159 m2,Yes,3,True
1,Exp_62,2020-02-01 01:00:00+00:00,2020-02-01,2,Phase_1,2.002,8.2,120-159 m2,Yes,3,True
2,Exp_62,2020-02-01 02:00:00+00:00,2020-02-01,3,Phase_1,2.019,8.4,120-159 m2,Yes,3,True
3,Exp_62,2020-02-01 03:00:00+00:00,2020-02-01,4,Phase_1,2.205,8.4,120-159 m2,Yes,3,True
4,Exp_62,2020-02-01 04:00:00+00:00,2020-02-01,5,Phase_1,2.834,8.2,120-159 m2,Yes,3,True


75/25

In [7]:
y=phase_1['Demand_kWh']
X= phase_1[["From","Temperature","home_size","electric_car","electrically_heated"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)
X_train.head()

,From,Temperature,home_size,electric_car,electrically_heated
0,2020-02-01 00:00:00+00:00,7.6,120-159 m2,Yes,True
1,2020-02-01 01:00:00+00:00,8.2,120-159 m2,Yes,True
2,2020-02-01 02:00:00+00:00,8.4,120-159 m2,Yes,True
3,2020-02-01 03:00:00+00:00,8.4,120-159 m2,Yes,True
4,2020-02-01 04:00:00+00:00,8.2,120-159 m2,Yes,True


In [8]:

model = auto_arima(y_train,
                   seasonal=True,
                   exogenous=X_train, 
                   suppress_warnings=True, 
                   error_action="ignore")

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                48600
Model:               SARIMAX(2, 1, 5)   Log Likelihood              -56002.850
Date:                Fri, 05 Jul 2024   AIC                         112021.700
Time:                        00:03:26   BIC                         112092.031
Sample:                             0   HQIC                        112043.760
                              - 48600                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.6438      0.003    513.903      0.000       1.638       1.650
ar.L2         -0.9198      0.003   -313.507      0.000      -0.926      -0.914
ma.L1         -2.0191      0.005   -444.361      0.0

In [9]:
n_periods = len(y_test)

forecast = model.predict(n_periods=n_periods, exogenous=X_test)
forecast.name = "forecast_data"
forecast_df = phase_1.join(forecast,how="left",rsuffix="_forecast")

print("Predicted values:", forecast)

mse = mean_squared_error(y_test, forecast)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, forecast)
print("Mean Absolute Error:", mae)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Predicted values: 48600    3.013734
48601    2.808083
48602    2.883354
48603    3.001698
48604    3.150471
           ...   
64795    3.146303
64796    3.146303
64797    3.146303
64798    3.146303
64799    3.146303
Name: forecast_data, Length: 16200, dtype: float64
Mean Squared Error: 4.583185598758996
Mean Absolute Error: 1.6763647818690621
Root Mean Squared Error: 2.1408375928031056


In [10]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Sarima/predictions/phase_1_with_survey_data/sarima_phase_1_survey_method_2_75_25.csv",index=True)

80 / 20 Ratio

In [6]:
y=phase_1['Demand_kWh']
X= phase_1[["From","Temperature","home_size","electric_car","electrically_heated"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
X_train.head()

,From,Temperature,home_size,electric_car,electrically_heated
0,2020-02-01 00:00:00+00:00,7.6,120-159 m2,Yes,Yes
1,2020-02-01 01:00:00+00:00,8.2,120-159 m2,Yes,Yes
2,2020-02-01 02:00:00+00:00,8.4,120-159 m2,Yes,Yes
3,2020-02-01 03:00:00+00:00,8.4,120-159 m2,Yes,Yes
4,2020-02-01 04:00:00+00:00,8.2,120-159 m2,Yes,Yes


In [7]:

model = auto_arima(y_train,
                   seasonal=True,
                   exogenous=X_train, 
                   suppress_warnings=True, 
                   error_action="ignore")

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                51840
Model:               SARIMAX(2, 1, 2)   Log Likelihood              -61811.783
Date:                Mon, 24 Jun 2024   AIC                         123633.566
Time:                        01:04:21   BIC                         123677.845
Sample:                             0   HQIC                        123647.413
                              - 51840                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4050      0.052      7.774      0.000       0.303       0.507
ar.L2          0.1047      0.033      3.155      0.002       0.040       0.170
ma.L1         -0.7172      0.052    -13.838      0.0

In [8]:
n_periods = len(y_test)

forecast = model.predict(n_periods=n_periods, exogenous=X_test)
forecast.name = "forecast_data"
forecast_df = phase_1.join(forecast,how="left",rsuffix="_forecast")

print("Predicted values:", forecast)

mse = mean_squared_error(y_test, forecast)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, forecast)
print("Mean Absolute Error:", mae)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Predicted values: 51840    3.659116
51841    3.864583
51842    3.977962
51843    4.045392
51844    4.084571
           ...   
64795    4.139698
64796    4.139698
64797    4.139698
64798    4.139698
64799    4.139698
Name: forecast_data, Length: 12960, dtype: float64
Mean Squared Error: 7.661067454593768
Mean Absolute Error: 2.3545304953842376
Root Mean Squared Error: 2.767863337412772


In [10]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Sarima/predictions/Phase 1 with survey data/sarima_phase_1_survey_method_2_80_20.csv",index=True)

60 / 40 Ratio

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)
X_train.head()

,From,Temperature,home_size,electric_car,electrically_heated
0,2020-02-01 00:00:00+00:00,7.6,120-159 m2,Yes,Yes
1,2020-02-01 01:00:00+00:00,8.2,120-159 m2,Yes,Yes
2,2020-02-01 02:00:00+00:00,8.4,120-159 m2,Yes,Yes
3,2020-02-01 03:00:00+00:00,8.4,120-159 m2,Yes,Yes
4,2020-02-01 04:00:00+00:00,8.2,120-159 m2,Yes,Yes


In [12]:

model = auto_arima(y_train,
                   seasonal=True,
                   exogenous=X_train, 
                   suppress_warnings=True, 
                   error_action="ignore")

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                38880
Model:               SARIMAX(2, 1, 2)   Log Likelihood              -46744.646
Date:                Mon, 24 Jun 2024   AIC                          93499.292
Time:                        01:07:51   BIC                          93542.134
Sample:                             0   HQIC                         93512.871
                              - 38880                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3747      0.070      5.391      0.000       0.238       0.511
ar.L2          0.0916      0.042      2.172      0.030       0.009       0.174
ma.L1         -0.7205      0.069    -10.413      0.0

In [13]:
n_periods = len(y_test)

forecast = model.predict(n_periods=n_periods, exogenous=X_test)
forecast.name = "forecast_data"
forecast_df = phase_1.join(forecast,how="left",rsuffix="_forecast")

print("Predicted values:", forecast)

mse = mean_squared_error(y_test, forecast)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, forecast)
print("Mean Absolute Error:", mae)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Predicted values: 38880    1.386980
38881    1.708181
38882    1.853826
38883    1.937828
38884    1.982649
           ...   
64795    2.036235
64796    2.036235
64797    2.036235
64798    2.036235
64799    2.036235
Name: forecast_data, Length: 25920, dtype: float64
Mean Squared Error: 3.3959806537074564
Mean Absolute Error: 1.4482561156725355
Root Mean Squared Error: 1.842818670870104


In [14]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Sarima/predictions/Phase 1 with survey data/sarima_phase_1_survey_method_2_60_40.csv",index=True)

50 / 50 Ratio

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, shuffle=False)
X_train.head()

,From,Temperature,home_size,electric_car,electrically_heated
0,2020-02-01 00:00:00+00:00,7.6,120-159 m2,Yes,Yes
1,2020-02-01 01:00:00+00:00,8.2,120-159 m2,Yes,Yes
2,2020-02-01 02:00:00+00:00,8.4,120-159 m2,Yes,Yes
3,2020-02-01 03:00:00+00:00,8.4,120-159 m2,Yes,Yes
4,2020-02-01 04:00:00+00:00,8.2,120-159 m2,Yes,Yes


In [16]:

model = auto_arima(y_train,
                   seasonal=True,
                   exogenous=X_train, 
                   suppress_warnings=True, 
                   error_action="ignore")

print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                32400
Model:               SARIMAX(2, 1, 5)   Log Likelihood              -40470.684
Date:                Mon, 24 Jun 2024   AIC                          80957.368
Time:                        01:11:22   BIC                          81024.455
Sample:                             0   HQIC                         80978.815
                              - 32400                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.6944      0.002    717.117      0.000       1.690       1.699
ar.L2         -0.9648      0.002   -425.155      0.000      -0.969      -0.960
ma.L1         -2.0849      0.005   -428.964      0.0

In [17]:
n_periods = len(y_test)

forecast = model.predict(n_periods=n_periods, exogenous=X_test)
forecast.name = "forecast_data"
forecast_df = phase_1.join(forecast,how="left",rsuffix="_forecast")

print("Predicted values:", forecast)

mse = mean_squared_error(y_test, forecast)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, forecast)
print("Mean Absolute Error:", mae)

rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Predicted values: 32400    3.379051
32401    3.221782
32402    3.152823
32403    3.182417
32404    3.280650
           ...   
64795    3.440121
64796    3.440121
64797    3.440121
64798    3.440121
64799    3.440121
Name: forecast_data, Length: 32400, dtype: float64
Mean Squared Error: 4.832084028440449
Mean Absolute Error: 1.8604104163409247
Root Mean Squared Error: 2.198200179337735


In [18]:
df = forecast_df[forecast_df["forecast_data"].notna()]
df = df.drop(columns=["Temperature","Hour","Participation_Phase"])

df.to_csv(parent_directory+"/Sarima/predictions/Phase 1 with survey data/sarima_phase_1_survey_method_2_50_50.csv",index=True)